In [ ]:
%reset
%load_ext autoreload
%autoreload 2

import time

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import defmod as dm

torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
source_image = dm.sampling.load_greyscale_image("../data/hippo_a.png")
target_image = dm.sampling.load_greyscale_image("../data/hippo_b.png")

In [ ]:
#aabb = dm.usefulfunctions.AABB(50., 150., 75., 150.)
aabb = dm.usefulfunctions.AABB(0., source_image.shape[0], 0., source_image.shape[1])
sigma = 5.
sigma_2 = 8.
x, y = torch.meshgrid([torch.arange(aabb.xmin, aabb.xmax, step=sigma), torch.arange(aabb.ymin, aabb.ymax, step=sigma)])
x_2, y_2 = torch.meshgrid([torch.arange(aabb.xmin, aabb.xmax, step=sigma_2), torch.arange(aabb.ymin, aabb.ymax, step=sigma_2)])

gd = dm.usefulfunctions.grid2vec(x, y).contiguous().view(-1)
gd_2 = dm.usefulfunctions.grid2vec(x_2, y_2).contiguous().view(-1)

trans = dm.deformationmodules.Translations(2, gd.view(-1, 2).shape[0], sigma)
trans_2 = dm.deformationmodules.Translations(2, gd_2.view(-1, 2).shape[0], sigma_2)

In [ ]:
plt.imshow(source_image)
plt.scatter(gd.view(-1, 2)[:, 0].numpy(), gd.view(-1, 2)[:, 1].numpy())
#plt.scatter(gd_2.view(-1, 2)[:, 0].numpy(), gd_2.view(-1, 2)[:, 1].numpy())
plt.show()

In [ ]:
my_model = dm.models.ModelCompoundImageRegistration(2, source_image, [trans], [gd], [True])
start_time = time.clock()
costs = my_model.fit(target_image, lr=0.001, l=150., max_iter=400, log_interval=10)
print("Elapsed time:", time.clock() - start_time)

In [ ]:
it = 5
out_gd, _ = my_model.shoot_list(it=it, intermediate=True)
sampled_out = my_model(it=it, intermediate=True)
grid_x, grid_y = my_model.compute_deformation_grid(torch.tensor([0., 0.]), torch.tensor([32., 32.]), torch.Size([10, 10]), it=it, intermediate=True)
%matplotlib qt5
for i in range(0, it):
    ax = plt.subplot(1, it+1, i+1)
    plt.imshow(sampled_out[i].detach().numpy(), cmap='gray')
    dm.usefulfunctions.plot_grid(ax, grid_x[i].detach().numpy(), -grid_y[i].detach().numpy()+32, color='C0')
    plt.plot(out_gd[i][0].view(-1, 2)[:, 0].detach().numpy(), -out_gd[i][0].view(-1, 2)[:, 1].detach().numpy()+32, '.')

plt.subplot(1, it+1, it+1)
plt.imshow(target_image, cmap='gray')

plt.show()

In [ ]:
plt.plot(range(0, len(costs)), costs)
plt.show()